In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from datetime import datetime

In [ ]:
tweets = pd.read_json("/Users/sam/Documents/Masters/Data_Systems_Project/Locations/user_locations.json")

In [ ]:
def split(x):
    x = str(x).split()[0]
    return x
tweets['created_at'] = tweets['created_at'].map(split)
tweets['created_at'] = pd.to_datetime(tweets['created_at'], format="%Y-%m-%d")

### Import protest data

In [ ]:
protest = pd.read_csv('ACLEDNeatAdress.csv')
protest['protest'] = 1
protest = protest.rename(columns={"EVENT_DATE": "created_at", "state2": "state", 'place': 'city'})
protest['created_at'] = pd.to_datetime(protest['created_at'], format="%Y-%m-%d")

In [ ]:
splitted=tweets["user_location"].str.split(", ", n = 1, expand = True)
tweets["state"] = splitted[1]
tweets["city"] = splitted[0]

In [ ]:
protest = protest.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'LATITUDE', 'LONGITUDE', 'road'])

In [ ]:
merged.head()

In [ ]:
tweets['created_at'] = tweets['created_at'].astype(str)
protest['created_at'] = protest['created_at'].astype(str)

merged = pd.merge(tweets, protest,  how='left', on=['created_at','state','city'])
merged = merged.fillna(0)

In [ ]:
merged = merged.groupby(['created_at','city']).size().reset_index(name='counts')

In [ ]:
mean = merged[['city', 'counts']].groupby('city').mean()
sd = merged[['city', 'counts']].groupby('city').std()

In [ ]:
merged = pd.merge(merged, mean,  how='left', on=['city'])


In [ ]:
merged = merged.rename(columns={"counts_y": "mean", 'counts_x': 'counts'})


In [ ]:
merged = pd.merge(merged, sd,  how='left', on=['city'])


In [ ]:
merged = merged.rename(columns={"counts_x": "counts", "counts_y": "sd"})


In [ ]:
merged.head()

In [ ]:
merged = merged[merged.counts>5]
merged = pd.merge(merged, protest,  how='left', on=['created_at','city'])
merged = merged.fillna(0)

In [ ]:
merged.head()

In [ ]:
merged['norm'] = (merged.counts-merged.mean)/merged.sd

In [ ]:
import statsmodels.api as sm 
    
# defining the dependent and independent variables 
Xtrain = [merged[['counts']], pd.Categorical(merged.city)]
ytrain = merged[['protest']]
   
# building the model and fitting the data 
log_reg = sm.Logit(ytrain, Xtrain).fit()

In [ ]:
print(log_reg.summary()) 

In [ ]:
crowd counting
number of people in protest
weekday effects
account for days on which verdicts and important new stories aired


In [ ]:
import numpy as np
import pandas as pd
from linearmodels import PanelOLS
from linearmodels import RandomEffects

In [ ]:
city = pd.Categorical(tweets.city)

In [ ]:
exog_vars = ['counts', 'city']
exog = sm.add_constant(tweets[exog_vars])
mod = PanelOLS(data.clscrap, exog)
re_res = mod.fit()
print(re_res)